<a href="https://colab.research.google.com/github/akshpesa/FMML/blob/main/FMML_M1L3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.datasets import mnist
from scipy.ndimage import rotate
from matplotlib import pyplot as plt

# Load MNIST data
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# Normalize the data
train_X = train_X / 255
test_X = test_X / 255

# Subsample to make computation faster (taking 50 images of each digit)
indices = np.hstack([np.where(train_y == i)[0][:50] for i in range(10)])
train_X = train_X[indices]
train_y = train_y[indices]

# Function to shear an image (basic version using shift in pixel rows/columns)
def shear(image, shear_val):
    shear_matrix = np.array([[1, shear_val], [0, 1]])
    tform = np.array([[i + shear_val * j for i, j in enumerate(row)] for row in image])
    return tform

# Function to augment images using rotation and shear
def augRotateShear(sample, angleconstraint, shearconstraint):
    if len(sample.shape) == 2:
        sample = np.expand_dims(sample, 0)
    amt = np.random.random(len(sample)) * shearconstraint
    amt = (amt - 0.5) * shearconstraint
    angle = np.random.random(len(sample)) * angleconstraint
    angle = (angle - 0.5) * angleconstraint
    nsample = sample.copy()
    for ii in range(len(sample)):
        nsample[ii] = rotate(shear(sample[ii], amt[ii]), angle[ii], reshape=False)
    return np.squeeze(nsample)

# NN classifier to find nearest neighbor
def NN1(traindata, trainlabel, query):
    diff = traindata - query
    sq = diff * diff
    dist = sq.sum(1)
    label = trainlabel[np.argmin(dist)]
    return label

def NN(traindata, trainlabel, testdata):
    traindata = traindata.reshape(-1, 28*28)
    testdata = testdata.reshape(-1, 28*28)
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

def Accuracy(gtlabel, predlabel):
    correct = (gtlabel == predlabel).sum()
    return correct / len(gtlabel)

# Parameters
shearconstraints = [0, 0.2, 0.4, 0.6, 0.8]
angleconstraints = [0, 10, 20, 30, 40]

# Initialize accuracies
accuracies = np.zeros((10, len(shearconstraints), len(angleconstraints)), dtype=float)

# Perform augmentation and calculate accuracy
for digit in range(10):
    digit_indices = np.where(train_y == digit)[0]
    digit_X = train_X[digit_indices]
    digit_y = train_y[digit_indices]

    for si, shear_val in enumerate(shearconstraints):
        for ai, angle_val in enumerate(angleconstraints):
            augdata = digit_X
            auglabel = digit_y

            # Augment the dataset
            naugmentations = 2
            for _ in range(naugmentations):
                augdata = np.concatenate((augdata, augRotateShear(digit_X, angle_val, shear_val)))
                auglabel = np.concatenate((auglabel, digit_y))  # Labels remain unchanged

            # Check test accuracy using NN
            testpred = NN(augdata, auglabel, test_X)
            accuracies[digit, si, ai] = Accuracy(test_y, testpred)

            print(
                f"Digit: {digit} - Accuracy after augmentation shear: {shear_val} angle: {angle_val} is {accuracies[digit, si, ai] * 100:.2f}%"
            )



Digit: 0 - Accuracy after augmentation shear: 0 angle: 0 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0 angle: 10 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0 angle: 20 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0 angle: 30 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0 angle: 40 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.2 angle: 0 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.2 angle: 10 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.2 angle: 20 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.2 angle: 30 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.2 angle: 40 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.4 angle: 0 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.4 angle: 10 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.4 angle: 20 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0.4 angle: 30 is 9.80%
Digit: 0 - Accuracy after augmentation shear: 0